# TP2
## Grupo 15

Carlos Eduardo Da Silva Machado A96936

Gonçalo Manuel Maia de Sousa A97485

## Problema 2

### Descrição do Problema

### Abordagem do Problema

## Código Python

In [1]:

from pysmt.shortcuts import *
from pysmt.typing import INT
import random as rn
N = 10
p = 0.01
centro = (4,4)
k = 10
def gera_rand():
    return rn.choices([0,1],[1-p,p],k=1)[0]

print(rn.choices([0,1],[1-p,p],k=1)[0])



0


In [2]:
def declare(i):
    state = dict()
    #state['pc'] = Symbol('pc'+str(i),INT)
    for i in range(1,N+1):
        for j in range(1,N+1):
            state[f'({i},{j})'] = Symbol('(i,j)'+str(i),INT)
    return state

def init(state):
    l = list()
    for i in range(1,N+1):
        l.append(Equals(state[f'({i},1)'], Int(gera_rand())))
    for j in range(2,N+1):
        l.append(Equals(state[f'(1,{j})'], Int(gera_rand())))
    
    ic,jc = centro
    
    for i in [ic-1,ic, ic+1]:
        for j in [jc-1, jc, jc+1]:
            l.append(Equals(state[f'({i},{j})'], Int(1)))
    return And(l)

def PontosProximos(pos):
    return [(pos[0]+1,pos[1]),(pos[0]-1,pos[1]),(pos[0],pos[1]+1),(pos[0],pos[1]-1),(pos[0]-1,pos[1]-1),(pos[0]+1,pos[1]+1),(pos[0]-1,pos[1]+1),(pos[0]+1, pos[1]-1)]

def trans(curr,prox):
    l = []
    for i in range(2,N+1):
        for j in range(2,N+1):
            valor = sum(filter(lambda x, y: x < N+1 and y < N+1 and (abs(x+i) == 1 or abs(y+j) == 1),  PontosProximos((i,j))))
            tBorn = And(Equals(curr[f'({i},{j})'], Int(0)), Equals(Int(valor), Int(3)), Equals(prox[f'({i},{j})'], Int(1)))
            tStayAlive = And(Equals(curr[f'({i},{j})'], Int(1)), Or(Equals(Int(valor), Int(2)),Equals(Int(valor), Int(3))), Equals(prox[f'({i},{j})'], curr[f'({i},{j})']))
            tDie = And(Equals(curr[f'({i},{j})'], Int(1)), Or(Int(valor) < Int(2),Int(valor) > Int(3)), Equals(prox[f'({i},{j})'], Int(0)))
            tStillDead = And(Equals(curr[f'({i},{j})'], Int(0)), Or(Int(valor) < Int(2),Int(valor) > Int(3)), Equals(prox[f'({i},{j})'],curr[f'({i},{j})']))
            l.add(Or(tBorn, tStayAlive, tDie, tStillDead))
            
    return And(l)



In [3]:
with Solver(name="msat") as s:
    
        # completar
    
        # cria k copias do estado
        trace = [declare(i) for i in range(k)]
        #print(trace)
        # criar o traço
        s.add_assertion(init(trace[0]))
        #print(init(trace[0]))
        for i in range(k-1):
            s.add_assertion(trans(trace[i], trace[i+1]))
        
        if s.solve():
            for i in range(k):
                print(i)
                print()
        else:
            print('no')

TypeError: trans.<locals>.<lambda>() missing 1 required positional argument: 'y'